# An Introduction to the Discrete Fourier Transform

# Lecture 4: Understanding the DFT SOLUTIONS

## The following is the SOLUTIONS to the exercises for Lecture 4 on the Discrete Fourier Transform. See this lecture video [here](https://www.youtube.com/watch?v=V0y9Rmq3jxw). For more supplemental resources, see also the course [website](https://longbaonguyen.github.io/courses/dft/discrete_fourier.html). 



## Exercises


In [3]:
import numpy as np
from IPython.display import Audio
import matplotlib.pyplot as plt
from scipy.io import wavfile
%matplotlib notebook

#### We consider the following signal in Lecture 4: $y(t)=3\sin(2\pi\cdot 10t)+2\cos(2\pi\cdot 20t+\pi/4), t\in [0,1/10]$.

#### Let's sample this signal at the sampling rate of `fs = 80` Hz for `L=1/10` second. Initialize the variables `fs`, `L` and `N`: total samples. (If you're using $N=L\cdot f_s$, remember to cast $N$ into an integer: `N=int(L*fs)`.)

In [4]:
fs = 80
L = 0.1
N = int(L*fs)


#### Create the time samples array `ts` and the samples `ys`. Use `np.linspace(start, stop, num, endpoint=False)` to inititalize `ts`.

In [5]:
ts = np.linspace(0,L,N,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)


#### We will now compute the Fourier coefficients of the above samples by using the formula

$$Y[k] = \displaystyle\sum_{n=0}^{N-1} y[n]e^{-i2\pi kn/N},k=0,1,…,N-1.$$

#### We will then generalize this and write our own function that compute the Discrete Fourier Transform. 

#### Let's use the formula above to compute $Y[1] =\displaystyle\sum_{n=0}^{7} y[n]e^{-i2\pi n/8}$. Use a for loop to sum up the term by term product. Store your answer in the variable `y1`. 

In [6]:
y1 = 0
for i in range(ys.size):
    y1 += ys[i]*np.exp(-2j*np.pi*i/8)    

#### Run the code below to check that your answer is `-12j`. Remember that the variable `j` is equal to the imaginary number `i`. 

In [7]:
np.allclose(y1, -12j)

True

#### To find all of the Fourier coefficients, use another loop, index by $k=0,1,2,...,7$. Thus, the Discrete Fourier Transform is simply a nested loop. First create an empty list. Call it `yk`. Then at each iteration, append a Fourier coefficient to the list. 

In [10]:
yk = []
for k in range(8):
    s = 0
    for n in range(8):
        s += ys[n]*np.exp(-1j*2*np.pi*k*n/8)
    yk.append(s)

In [11]:
yk

[(8.43769498715119e-15+0j),
 (2.7200464103316335e-15-11.999999999999995j),
 (5.656854249492379+5.656854249492385j),
 (-1.6653345369377348e-15-4.996003610813204e-16j),
 (-4.884981308350689e-15-7.768113139884329e-16j),
 (5.495603971894525e-15+8.881784197001252e-16j),
 (5.656854249492392-5.656854249492378j),
 (-4.718447854656915e-15+11.999999999999996j)]

#### Run the following code to make sure your result `yk` gives the same result as `np.fft.fft()`. `np.allclose()` returns True if two arrays are element-wise equal within a tolerance.

In [15]:
np.allclose(yk, np.fft.fft(ys)) # should return True


True

#### Now combine all of the above code to write the function `dft` which accepts an array of samples and returns a list of Fourier coefficients. Use a nested for loop.

In [32]:
def dft(ys):
    """
    Converts an array of samples ys to a list of Fourier coefficients. Use a nested for loop. 
    """
    N = ys.size
    yk = []
    for k in range(N):
        s = 0
        for n in range(N):
            s += ys[n]*np.exp(-1j*2*np.pi*n*k/N)
        yk.append(s)
    return yk
    

#### Run the following code to make sure `dft` gives the same result as `np.fft.fft()`. `np.allclose()` returns True if two arrays are element-wise equal within a tolerance.

In [17]:
fs = 80
L = .1
N = int(L*fs)
ts = np.linspace(0,L,N,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)

np.allclose(dft(ys), np.fft.fft(ys)) # should return True


True

#### Unfortunately, the above code is very slow! Let's compare this `dft` version with `np.fft.fft` by timing each function. Run the following code. My 2017 MacAir took 31.95 seconds on a 0.06 second audio.

In [25]:
fs = 44100
L = .06
N = int(L*fs)
ts = np.linspace(0,L,N,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)

from time import clock
start = clock()
yk = dft(ys)
elapsed = clock() - start
print("Time elapsed:", elapsed, "seconds.")


Time elapsed: 30.552117000000003 seconds.


#### Now let's see how much time np.fft.fft takes. (0.00212 second!, about 15000 times faster!) In general, the nested loop version of DFT takes $N^2$ operations while the FFT takes $N\log(N)$. This is a huge difference as $N$ gets larger. This is similar to the difference between selection sort versus mergesort.

In [19]:
start = clock()
yk = np.fft.fft(ys)
elapsed = clock() - start
print("Time elapsed:", elapsed, "seconds.")

Time elapsed: 0.0027889999999999304 seconds.


#### We can write faster version of the DFT with Numpy's optimized operations instead of using a nested loop.

#### First, compute the array of numbers for the index `n = [0,1,2...,7]`. Use `np.arange(8)` function and store it in the variable `n`.

In [29]:
n = np.arange(8)
ts = np.linspace(0,1,8,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)


#### Then apply the exponential function to the array `n`. Use `np.exp()`. Store this array in the variable `exp`.

In [30]:
exp = np.exp(-1j*2*np.pi*n*2/8)


#### Multiply the array `ys` with the array `exp`. Numpy will multiply these arrays elementwise. Then use the built-in `sum()` function to find the sum of the resulting dot product. This sum is the coefficient `Y[1]`. Check that your answer is `-12j`. This computation uses Numpy's optimized operations and is much faster than using a for loop as done in the function `dft` above.

In [31]:
sum(ys*exp)

(-3.585553445878575e-14-12.000000000000025j)

#### Now combine all of the above code to write the function `dft2` which uses Numpy's optimized operations. This function should use only one for loop. 

In [33]:
def dft2(ys):
    """
    Converts an array of samples to an array of Fourier coefficients. Uses one for loop and Numpy's optimized operations.
    """
    N = ys.size
    yk = []
    for k in range(N):
        n = np.arange(N)
        exp = np.exp(-1j*2*np.pi*n*k/N)
        yk.append(sum(ys*exp))
    return yk

#### Check to see if `dft` and `dft2` give the same result as `np.fft.fft`.

In [34]:
fs = 100
L = .1
N = int(L*fs)
ts = np.linspace(0,L,N,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)
print("dft is the same as dft2: ", np.allclose(dft(ys),dft2(ys)))
print("dft2 is the same as np.fft.fft: ", np.allclose(dft2(ys),np.fft.fft(ys)))

dft is the same as dft2:  True
dft2 is the same as np.fft.fft:  True


#### Let's compare dft and dft2 runnning times on 0.6 audio clip. `dft`: 31.95 seconds, `dft2`: 1.008 seconds.

In [72]:
fs = 44100
L = .06
N = int(L*fs)
ts = np.linspace(0,L,N,endpoint=False)
ys = 3*np.sin(2*np.pi*10*ts) + 2*np.cos(2*np.pi*20*ts+np.pi/4)

from time import clock
start = clock()
yk = dft2(ys)
elapsed = clock() - start
print("Time elapsed:", elapsed, "seconds.")


Time elapsed: 1.007641000000035 seconds.
